In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import numpy.typing as npt
import csv
from tkinter import Tk
from tkinter import filedialog
from dataclasses import dataclass

In [2]:
def smilesToMolecule(smilesString: str):
    molecule = Chem.AddHs(Chem.MolFromSmiles(smilesString)) 
    AllChem.EmbedMolecule(molecule, AllChem.ETKDGv3())  # Puts the molecule into 3 dimensions

    xyzString = Chem.rdmolfiles.MolToXYZBlock(molecule)
    moleculeXYZ = [i.split() for i in xyzString.split("\n")[2:-1]]

    # List of all atomic symbols in molecule, in order from RDkit generated XYZ file
    atomIdentities = [str(i[0]) for i in moleculeXYZ]
    # Array with all atomic coordinates, without atomic symbols
    atomCoords = np.array([[float(j) for j in i[1:]] for i in moleculeXYZ])

    return atomIdentities, atomCoords

In [3]:
def substructureMatch(smilesString: str) -> tuple[int, list[int]]:
    """Generate an RDKit molecule and search the structure for protonated atoms."""
    
    molecule = Chem.AddHs(Chem.MolFromSmiles(smilesString))
    params = AllChem.ETKDGv3()
    AllChem.EmbedMolecule(molecule, params)

    chargedAtom = Chem.MolFromSmarts(
        '[#7H+,#7H2+,#7H3+,#8H+,#8H2+]'
    )
    
    protonAtom = Chem.MolFromSmarts(
        '[$([#1][#7H+]),$([#1][#7H2+]),$([#1][#7H3+]),$([#1][#8H+]),$([#1][#8H2+])]'
    )
    
    protonPosition = molecule.GetSubstructMatches(protonAtom)
    parentPosition = molecule.GetSubstructMatches(chargedAtom)

    protonatedAtomIndex = parentPosition[0][0]
    protonPositionIndices = [i[0] for i in protonPosition]

    return protonatedAtomIndex, protonPositionIndices

In [ ]:
class Molecule:
    """Class that contains all molecule data"""

    def __init__(self, smilesString: str, **kwargs):
        self.smilesString = smilesString
        if not kwargs:
            self.atomIdentities, self.atomCoords = smilesToMolecule(self.smilesString)
            self.protonatedAtomIndex, self.protonPositionIndices = substructureMatch(self.smilesString)
            self.translatedAtomCoords = self.atomCoords - self.atomCoords[self.protonatedAtomIndex]
        else:
            self.protonatedAtomIndex: int = kwargs['protonatedAtomIndex']
            self.protonPositionIndices: list[int] = kwargs['protonPositionIndices']
            self.moleculeNumber: str = kwargs['moleculeNumber']
            self.atomIdentities: list[str] = kwargs['atomIdentities']
            self.atomCoords: npt.NDArray = kwargs['atomCoords']
            self.translatedAtomCoords: npt.NDArray = self.atomCoords - self.atomCoords[self.protonatedAtomIndex]
        
    def getProtonPosition(self, attemptNumber) -> npt.NDArray:
        return self.translatedAtomCoords[int(self.protonPositionIndices[attemptNumber])]
    
    def getProtonatedAtomPosition(self) -> npt.NDArray:
        return self.translatedAtomCoords[self.protonatedAtomIndex]
    
    def getNumAtoms(self) -> int:
        return len(self.atomIdentities)

In [5]:
smilesString = 'N#Cc1[nH][nH+]nc(C#N)1'
molecule = Chem.AddHs(Chem.MolFromSmiles(smilesString)) 
params = AllChem.ETKDGv3()
AllChem.EmbedMolecule(molecule, params)  # Puts the molecule into 3 dimensions

xyzString = Chem.rdmolfiles.MolToXYZBlock(molecule)

numAtoms = int(xyzString.split()[0])
moleculeXYZ = [i.split() for i in xyzString.split("\n")[2:-1]]

# List of all atomic symbols in molecule, in order from RDkit generated XYZ file
atomIdentities = [str(i[0]) for i in moleculeXYZ]
# Array with all atomic coordinates, without atomic symbols
atomCoords = np.array([[float(j) for j in i[1:]] for i in moleculeXYZ])

In [6]:
# Set commonly used variables

zRotationOffset = np.array(
    [
        [-1, 0, 0], 
        [0, -1, 0], 
        [0,  0, 1]
    ],
    dtype=float
)

# Array of values for the distances between parent atoms
DISTANCE_MULTIPLIERS = [2.7, 2.8, 2.9, 3.0, 3.1, 3.2]

Z_UNIT_VECTOR = np.array([0.0, 0.0, 1.0])

Z_REFLECTOR = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0,-1]
    ],
    dtype=float)


In [7]:
def multipleInput() -> tuple[list, str]:
    """Read CSV files with multiple molecules and specify folder containing XYZ files"""

    Tk().withdraw()
    filename = filedialog.askopenfilename()

    multipleMoleculeData = []

    with open(filename, newline='') as csvfile:
        data = csv.reader(csvfile)
        for row in data:
            multipleMoleculeData.append(row)

    Tk().withdraw()
    xyzFilePath = filedialog.askdirectory() + '/'

    return multipleMoleculeData[1:], xyzFilePath

In [8]:
def normalize(vector: npt.NDArray) -> npt.NDArray:
    """Take a vector and return the corresponding unit vector."""

    norm = np.linalg.norm(vector) 

    if norm != 0:
        return vector/norm
    else:
        return vector

In [9]:
def quaternionBuilder(
    vector: npt.ArrayLike,
    angle: npt.DTypeLike,
    ) -> npt.NDArray:
    """Build a quaternion matrix"""

    quaternion = [
        np.cos(angle/2), 
        vector[0] * np.sin(angle/2), 
        vector[1] * np.sin(angle/2), 
        vector[2] * np.sin(angle/2),
    ]

    # The normalized quaternion vector
    normQuat = normalize(quaternion)

    quaternionMatrix = np.array(
        [
            [
                1 - 2 * (normQuat[2] ** 2 + normQuat[3] ** 2),
                2 * (normQuat[1] * normQuat[2] - normQuat[0] * normQuat[3]),
                2 * (normQuat[1] * normQuat[3] + normQuat[0] * normQuat[2]),
            ],
            [
                2 * (normQuat[1] * normQuat[2] + normQuat[0] * normQuat[3]),
                1 - 2 * (normQuat[1] ** 2 + normQuat[3] ** 2),
                2 * (normQuat[2] * normQuat[3] - normQuat[0] * normQuat[1]),
            ],
            [
                2 * (normQuat[1] * normQuat[3] - normQuat[0] * normQuat[2]),
                2 * (normQuat[2] * normQuat[3] + normQuat[0] * normQuat[1]),
                1 - 2 * (normQuat[1] ** 2 + normQuat[2] ** 2),
            ],
        ]
    )

    return quaternionMatrix

In [10]:
def vectorAngle(vectorOne: npt.ArrayLike, vectorTwo: npt.ArrayLike) -> np.float64:
    """Calculate the angle (in radians) between two vectors"""

    unitVectorOne = normalize(vectorOne)
    unitVectorTwo = normalize(vectorTwo)

    angle = np.arccos(np.clip(np.dot(unitVectorOne, unitVectorTwo), -1.0, 1.0))

    return angle

In [11]:
def distanceCalculator(vectorOne: npt.ArrayLike, vectorTwo: npt.ArrayLike) -> np.float64 :
    """Calculate Euclidean distance between two vectors"""

    distance = np.absolute(
        np.sqrt(
            ((float(vectorOne[0]) - float(vectorTwo[0])) ** 2)
            + ((float(vectorOne[1]) - float(vectorTwo[1])) ** 2)
            + ((float(vectorOne[2]) - float(vectorTwo[2])) ** 2)
        )
    )

    return distance

In [12]:
def xyzFileRead(filePath: str) -> tuple[list[str], npt.NDArray[np.float64]]:
    """Read in XYZ file format and return atomic symbols and coordinates"""

    moleculeXYZ = []

    with open(filePath) as f:
        for line in f:
             # Takes each line in the file minus the last character, which is just the \n
            line = line[:-1].split()
            if line:
                # If the line isn't empty then append to the moleculeXYZ
                moleculeXYZ.append(line) 

    # List of all atomic symbols in molecule, in order from RDkit generated XYZ file
    atomIdentities = [str(i[0]) for i in moleculeXYZ[1:]]

    # Array with all atomic coordinates, without atomic symbols
    atomCoords = np.array([[float(j) for j in i[1:]] for i in moleculeXYZ[1:]])

    return atomIdentities, atomCoords

In [13]:
def nudgeMatrixGenerator(stage: str) -> tuple[npt.NDArray, npt.NDArray, npt.NDArray]:
    """Creates the nudge matrices and vector for avoiding local minima during optimization.

    `stage` should be either `"R"` (Reactant) or `"P"` (Product) or `"T"` (Transition State)
    """

    # xvDegree (15 Degrees)
    xvDegree = np.pi/12
    # xxDegree (20 Degrees)
    xxDegree = np.pi/9

    nudgeTranslate = np.array([0, 0, 0])
    
    nudgeRotateX = np.empty([3, 3], dtype=float)
    nudgeRotateY = np.empty([3, 3], dtype=float)

    if stage == 'R':
        nudgeRotateX = np.array(
            [
                [1, 0, 0],
                [0, np.cos(xvDegree), -np.sin(xvDegree)],
                [0, np.sin(xvDegree), np.cos(xvDegree)],
            ],
            dtype=float,
        )

        nudgeRotateY = np.array(
            [
                [np.cos(xxDegree), 0, np.sin(xxDegree)],
                [0, 1, 0],
                [-np.sin(xxDegree), 0, np.cos(xxDegree)],
            ],
            dtype=float,
        )
    elif stage == 'P':
        nudgeRotateX = np.array(
            [
                [1, 0, 0],
                [0, np.cos(-xvDegree), -np.sin(-xvDegree)],
                [0, np.sin(-xvDegree), np.cos(-xvDegree)],
            ],
            dtype=float,
        )

        nudgeRotateY = np.array(
            [
                [np.cos(-xxDegree), 0, np.sin(-xxDegree)],
                [0, 1, 0],
                [-np.sin(-xxDegree), 0, np.cos(-xxDegree)],
            ],
            dtype=float,
        )



    return nudgeRotateX, nudgeRotateY, nudgeTranslate

In [14]:
def checkStructureOneGenerator(
    moleculeAlignmentQuaternion: npt.NDArray, 
    mol: Molecule,
    ) -> list:
    """Align the molecule along Z axis and store its structure"""

    checkStructureOne = []

    for atomPosition in mol.translatedAtomCoords:
        newCoord = np.dot(moleculeAlignmentQuaternion, atomPosition)
        newCoordRounded = [float(round(number,6)) for number in newCoord]
        checkStructureOne.append(newCoordRounded.copy())

    return checkStructureOne

In [15]:
def checkStructureTwoGenerator(
    stage: str,
    distanceMultiplier: int,
    zRotationOffset: npt.NDArray,
    checkStructureOne: npt.ArrayLike,
    ) -> list:
    """Generate the second structure that the proton will be transferred to"""

    nudgeRotateX, nudgeRotateY, nudgeTranslate = nudgeMatrixGenerator(stage)
    checkStructureTwo = []

    for atomPosition in checkStructureOne:
        if stage == "R":
            newCoord = (
                np.dot(
                    nudgeRotateX,
                    (
                        np.dot(
                            nudgeRotateY,
                            (np.dot(zRotationOffset, (np.dot(Z_REFLECTOR, atomPosition))))
                            + (distanceMultiplier * Z_UNIT_VECTOR),
                        )
                    ),
                )
                + (nudgeTranslate)
            )
        elif stage == "P":
            newCoord = (
                np.dot(
                    nudgeRotateX,
                    (
                        np.dot(
                            nudgeRotateY,
                            (np.dot(zRotationOffset, (np.dot(Z_REFLECTOR, atomPosition))))
                            + (distanceMultiplier * Z_UNIT_VECTOR),
                        )
                    ),
                )
                + ((-1) * nudgeTranslate)
            )
        elif stage == "T":
            newCoord = np.dot(
                zRotationOffset,
                (np.dot(Z_REFLECTOR, atomPosition) + ((distanceMultiplier - 0.1) * Z_UNIT_VECTOR)),
            )
        newCoordRounded = [float(round(number,6)) for number in newCoord]
        checkStructureTwo.append(newCoordRounded.copy())

    return checkStructureTwo

In [16]:
def structureChecker(
    checkStructureOne: npt.ArrayLike,
    checkStructureTwo: npt.ArrayLike,
    mol: Molecule,
    attemptNumber: int,
    ) -> bool:
    """Check for overlap between the two molecules"""

    n = mol.protonPositionIndices[attemptNumber]
    for i, atomOne in enumerate(checkStructureOne):
        for j, atomTwo in enumerate(checkStructureTwo):
            if (i == n) or (j == n - 1):
                continue
            # Only executes if not proton involved in transport
            if distanceCalculator(atomOne, atomTwo) < 1.5:
                return True
            else:
                continue
    # We checked all atoms, none overlapped
    return False

In [17]:
def finalStructureGenerator(
    checkStructureOne: npt.ArrayLike, 
    checkStructureTwo: npt.ArrayLike, 
    distanceMultiplier: int,
    stage: str,
    mol: Molecule,
    attemptNumber: int,
    ) -> list:
    """Generate the complete structure"""

    newStructureOne = []
    newStructureTwo = []

    for i in range(len(checkStructureOne)):
        newStructureOne.append([atomIdentities[i]] + checkStructureOne[i])
        newStructureTwo.append([atomIdentities[i]] + checkStructureTwo[i])

    reactantProton = newStructureOne.pop(mol.protonPositionIndices[attemptNumber])
    productProton = newStructureTwo.pop(mol.protonPositionIndices[attemptNumber])
    transitionProton = ['H', 0.0, 0.0, (distanceMultiplier-0.1)/2]

    finalStructure = newStructureOne + newStructureTwo

    if stage == 'R':
        finalStructure.append(reactantProton)
    elif stage == 'P':
        finalStructure.append(productProton)
    elif stage == 'T':
        finalStructure.append(transitionProton)

    return finalStructure

In [27]:
def overlapHandler(
    checkStructureOne: npt.ArrayLike,
    checkStructureTwo: npt.ArrayLike,
    stage: str,
    distanceMultiplier: float,
    mol: Molecule,
    zRotationOffset: npt.NDArray,
    ) -> tuple[list, list, int]:
    """Fix molecule overlap issues"""

    protonPositionAttempts = len(mol.protonPositionIndices)
    attemptNumber = 0

    while structureChecker(checkStructureOne, checkStructureTwo, mol, attemptNumber) == True and attemptNumber < protonPositionAttempts:

        zRotationOffset = np.array(
            [
                [-1, 0, 0],
                [0, -1, 0],
                [0,  0, 1]
            ],
            dtype=float)

        protonPosition = mol.translatedAtomCoords[int(mol.protonPositionIndices[attemptNumber])]

        moleculeAlignmentAngle = vectorAngle(protonPosition, Z_UNIT_VECTOR)
        moleculeAlignmentVector = normalize(np.cross(protonPosition, Z_UNIT_VECTOR))
        moleculeAlignmentQuaternion = quaternionBuilder(moleculeAlignmentVector, moleculeAlignmentAngle)

        checkStructureOne = checkStructureOneGenerator(moleculeAlignmentQuaternion, mol.translatedAtomCoords)
        checkStructureTwo = checkStructureTwoGenerator(stage, distanceMultiplier, zRotationOffset, checkStructureOne)
        
        # If the structure works, then return it and exit the function
        if structureChecker(checkStructureOne, checkStructureTwo, mol, attemptNumber) == False:
            print('Structure was fixed with proton position #'+str(attemptNumber))
            return checkStructureOne, checkStructureTwo, attemptNumber

        i = 0
        while structureChecker(checkStructureOne, checkStructureTwo, mol, attemptNumber) == True and i < 8:

            rotationAngle = i * (np.pi/4)

            zRotationOffset = np.array(
                [
                    [np.cos(rotationAngle), -np.sin(rotationAngle), 0],
                    [np.sin(rotationAngle),  np.cos(rotationAngle), 0],
                    [0, 0, 1]
                ],
                dtype=float)
            
            checkStructureTwo = checkStructureTwoGenerator(stage, distanceMultiplier, zRotationOffset, checkStructureOne)

            if structureChecker(checkStructureOne, checkStructureTwo, mol, attemptNumber) == False:
                print('Structure was fixed with proton position #'+str(attemptNumber)+' with a rotation of '+str(i*45)+' degrees')
                return checkStructureOne, checkStructureTwo, attemptNumber

            i += 1
        
        attemptNumber += 1
    print('Structure could not be fixed.')
    return checkStructureOne, checkStructureTwo, attemptNumber

In [36]:
# This is for if you want to run structure generation from a CSV with multiple molecules

#
# DO NOT USE DURING TESTING
#

# stage = input('Input stage (R, P, T): ')
stage = 'R'

distanceMultiplier = DISTANCE_MULTIPLIERS[0]

multipleMoleculeData, xyzFilePath = multipleInput()
print(multipleMoleculeData[0][5])
for molecule in multipleMoleculeData:

    moleculeNumber = molecule[5]
    atomIdentities, atomCoords = xyzFileRead(xyzFilePath+moleculeNumber+'.xyz')

    molInformation = {
        'protonatedAtomIndex': int(molecule[1]),
        'protonPositionIndices': [int(i) for i in molecule[2:5] if i != 'NA'],
        'moleculeNumber': moleculeNumber,
        'atomIdentities': atomIdentities,
        'atomCoords': atomCoords,
    }

    bean = molecule[0]

    Structure = Molecule(
        'C#CCCC[NH2+]C',
        **molInformation
    )
    
    attemptNumber = 0

    protonPosition = Structure.getProtonPosition(attemptNumber)

    moleculeAlignmentAngle = vectorAngle(protonPosition, Z_UNIT_VECTOR)
    moleculeAlignmentVector = normalize(np.cross(protonPosition, Z_UNIT_VECTOR))
    moleculeAlignmentQuaternion = quaternionBuilder(moleculeAlignmentVector, moleculeAlignmentAngle)

    checkStructureOne = checkStructureOneGenerator(moleculeAlignmentQuaternion, Structure)
    checkStructureTwo = checkStructureTwoGenerator(stage, distanceMultiplier, zRotationOffset, checkStructureOne)

    print(moleculeNumber)

    attemptNumber = 0

    if structureChecker(checkStructureOne, checkStructureTwo, Structure, 0):
        print("PANIC")
        checkStructureOne, checkStructureTwo, attemptNumber = overlapHandler(
            checkStructureOne, 
            checkStructureTwo, 
            stage, 
            distanceMultiplier, 
            Structure, 
            zRotationOffset,
            )

    finalStructure = finalStructureGenerator(
        checkStructureOne,
        checkStructureTwo,
        distanceMultiplier,
        stage,
        Structure,
        attemptNumber
        )

    with open(str(moleculeNumber)+'-'+stage+'.xyz', 'w', newline='') as FinalFile: # creates a text file and specifies that it is supposed to be written to
        headerstart = str(Structure.getNumAtoms()*2-1) + '\n' # Determines the number of atoms and converts to a string
        np.savetxt(FinalFile, finalStructure, fmt='%s', header=headerstart,comments='') # Saves the numpy array as a txt file for reading later.



TestMolecule
{'protonatedAtomIndex': 5, 'protonPositionIndices': [14, 15], 'moleculeNumber': 'TestMolecule', 'atomIdentities': ['C', 'C', 'C', 'C', 'C', 'N', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'], 'atomCoords': array([[ 9.9910717 ,  7.8112166 ,  7.16364177],
       [ 9.09400807,  7.35389943,  6.48803389],
       [ 7.96058267,  6.80476725,  5.76419886],
       [ 6.6381651 ,  7.57747395,  5.99540307],
       [ 6.54745683,  8.30552501,  7.326568  ],
       [ 6.99746894,  7.4509756 ,  8.48654735],
       [ 6.19646185,  6.21854497,  8.72822711],
       [10.82880653,  8.21148282,  7.70938183],
       [ 7.84181548,  5.74023715,  6.05083243],
       [ 8.18793218,  6.7825582 ,  4.68206036],
       [ 6.51926278,  8.3573889 ,  5.22147008],
       [ 5.78838294,  6.88476141,  5.86073337],
       [ 5.51834638,  8.64179501,  7.54016444],
       [ 7.21722264,  9.18306727,  7.33924926],
       [ 8.00969391,  7.22352298,  8.28967507],
       [ 6.9982285 ,  8.03511088,  9.34397

AttributeError: 'Molecule' object has no attribute 'translatedAtomCoords'

In [37]:
# This is for if you want to run structure generation from a CSV with multiple molecules

stage = 'R'

distanceMultiplier = DISTANCE_MULTIPLIERS[0]

multipleMoleculeData, xyzFilePath = multipleInput()

protonatedAtomIndex = int(multipleMoleculeData[0][1])
protonPositionIndices = [int(i) for i in multipleMoleculeData[0][2:5] if i != 'NA']
moleculeNumber = multipleMoleculeData[0][5]

atomIdentities, atomCoords, numAtoms = xyzFileRead(xyzFilePath+moleculeNumber+'.xyz')

protonatedAtomPosition = atomCoords[protonatedAtomIndex] # Original position of parent atom
translatedAtomCoords = atomCoords - protonatedAtomPosition # Puts the parent atom at (0, 0, 0) and translates all other atoms too
protonPosition = translatedAtomCoords[int(protonPositionIndices[0])] # Coordinates for the proton involved in transport

moleculeAlignmentAngle = vectorAngle(protonPosition, Z_UNIT_VECTOR)
moleculeAlignmentVector = normalize(np.cross(protonPosition, Z_UNIT_VECTOR))
moleculeAlignmentQuaternion = quaternionBuilder(moleculeAlignmentVector, moleculeAlignmentAngle)

checkStructureOne = checkStructureOneGenerator(moleculeAlignmentQuaternion, translatedAtomCoords)
checkStructureTwo = checkStructureTwoGenerator(stage, distanceMultiplier, zRotationOffset, checkStructureOne.copy())

print(moleculeNumber)

if structureChecker(checkStructureOne, checkStructureTwo, protonPositionIndices, 0):
    print("PANIC")
    checkStructureOne, checkStructureTwo, attemptNumber = overlapHandler(
        checkStructureOne, 
        checkStructureTwo, 
        stage, 
        distanceMultiplier, 
        translatedAtomCoords, 
        protonPositionIndices, 
        zRotationOffset
    )

finalStructure = finalStructureGenerator(
    checkStructureOne, 
    checkStructureTwo, 
    distanceMultiplier, 
    stage, 
    protonPositionIndices, 
    attemptNumber
)

with open('Test-'+stage+'.xyz', 'w', newline='') as FinalFile:
    headerstart = str(numAtoms*2-1) + '\n'
    np.savetxt(FinalFile, finalStructure, fmt='%s', header=headerstart,comments='')

TestMolecule
PANIC
Structure was fixed with proton position #1
